In [1]:
import tqdm
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from PIL import ImageDraw, ImageFont
from skimage.measure import label, regionprops
import cv2
import os
from typing import Tuple
import random
from tqdm import tqdm

from src.sections import RegionDetector


def find_field_center(mask: np.ndarray, font_size) -> Tuple[int, int]:
    image_height = len(mask) - 1
    image_width = len(mask[0]) - 1

    padding = 5

    font_width, font_height = font_size

    font_width = font_width + (padding * 2)
    font_height = font_height + (padding * 2)


    def outside_image(index):
        index_y, index_x = index

        return (
                (index_y < 0) or (index_y > image_height) or
                (index_x < 0) or (index_x > image_width)
        )

    region_detector = RegionDetector()

    #print('get regions: start')
    regions = region_detector.get_regions(mask)
    #print('get regions: end')

    #print(mask.shape)
    #print('find position')
    for region in regions:
        for middle_pixel in  region.get_region_center_middle_positions():
            current_x, current_y = middle_pixel

            high_x_index = current_y, (current_x + int(font_width / 2))
            low_x_index = current_y, (current_x - int(font_width / 2))
            high_y_index = (current_y + int(font_height / 2)), current_x
            low_y_index = (current_y - int(font_height / 2)), current_x

            if (
                    (outside_image(high_x_index)) or
                    (outside_image(low_x_index)) or
                    (outside_image(high_y_index)) or
                    (outside_image(low_y_index))
            ):
                continue

            if (
                    (mask[high_x_index] == True) and
                    (mask[low_x_index] == True) and
                    (mask[high_y_index] == True) and
                    (mask[low_y_index] == True)
            ):
                return current_x, current_y

    #print('no position found')
    #print('no center found')
    return 0, 0


def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return np.array(image)


def quantize_colors(image_array, n_colors):
    # Apply Gaussian blur to smooth transitions and reduce small color noise
    blurred = cv2.GaussianBlur(image_array, (5, 5), 0)

    h, w, c = blurred.shape
    image_flat = blurred.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_colors, random_state=42).fit(image_flat)
    labels = kmeans.labels_
    quantized_flat = kmeans.cluster_centers_[labels].astype('uint8')
    quantized_image = quantized_flat.reshape(h, w, 3)
    return quantized_image, labels.reshape(h, w), kmeans.cluster_centers_


def detect_regions_by_color(quantized_image):
    h, w, _ = quantized_image.shape
    labeled_image = np.zeros((h, w), dtype=int)
    current_label = 1

    unique_colors = np.unique(quantized_image.reshape(-1, 3), axis=0)
    for color in unique_colors:
        mask = np.all(quantized_image == color, axis=-1)
        labeled_mask = label(mask, connectivity=1)
        labeled_mask[labeled_mask > 0] += current_label - 1
        labeled_image += labeled_mask
        current_label = labeled_image.max() + 1

    return labeled_image


def filter_small_regions(labeled_image, min_size=100):
    new_labeled = np.zeros_like(labeled_image)
    current_label = 1
    for region in regionprops(labeled_image):
        if region.area >= min_size:
            coords = region.coords
            for y, x in coords:
                new_labeled[y, x] = current_label
            current_label += 1
    return new_labeled


def assign_numbers(cluster_centers):
    color_to_number = {tuple(center.astype(int)): idx + 1 for idx, center in enumerate(cluster_centers)}
    return color_to_number


def save_legend(color_to_number, output_folder):
    legend_path = os.path.join(output_folder, "legend.txt")
    with open(legend_path, "w") as f:
        for color, number in color_to_number.items():
            f.write(f"{number}: RGB{color}\n")
    print(f"Legend saved to {legend_path}")


def overlay_numbers(
        image_array: np.ndarray,
        labeled_regions: np.ndarray,
        color_to_number: dict,
        font_path: str = None,
        font_size: int = 20,
        num_samples: int = 200
) -> Image.Image:
    image = Image.fromarray(image_array)
    draw = ImageDraw.Draw(image)
    if font_path:
        font = ImageFont.truetype(font_path, font_size)
    else:
        font = ImageFont.load_default()

    # Pro Region einmal durchgehen
    for region in tqdm(regionprops(  labeled_regions)):
        # War bereits vorher so im code. Ich nehme an, dass ist der Hintergrund?!
        if region.label == 0:
            continue

        mask_full = (labeled_regions == region.label)

        color = tuple(image_array[mask_full][0])
        number = color_to_number.get(color, '?')

        font_size = font.getsize(str(number))

        # Hier wird meine neue Funktion aufgerufen.
        cx, cy = find_field_center(mask_full, font_size)

        # Übernahme von Pascal seinem Code

        font_width, font_height = font_size

        draw.text((cx - int(font_width / 2), cy - int(font_height / 2)), str(number), fill='black', font=font)

    return image


def create_paint_by_numbers(image_path, save_path, n_colors):
    output_folder = save_path
    os.makedirs(output_folder, exist_ok=True)

    image_array = load_image(image_path)
    quantized_image, label_image, cluster_centers = quantize_colors(image_array, n_colors)


    labeled_regions = detect_regions_by_color(quantized_image)
    labeled_regions = filter_small_regions(labeled_regions, min_size=150)  # tweak size
    color_to_number = assign_numbers(cluster_centers)
    final_image = overlay_numbers(quantized_image, labeled_regions, color_to_number)

    # Save image
    output_image_path = os.path.join(output_folder, "paint_by_numbers_output.png")
    final_image.save(output_image_path)
    print(f"Paint-by-numbers image saved to {output_image_path}")

    # Save legend
    save_legend(color_to_number, output_folder)



In [ ]:
test = np.array([
    [False, False, True, True, True, True, True, False, False, False],
    [False, False, True, True, True, True, True, False, False, False],
    [False, False, True, True, True, True, True, False, False, False],
    [True, True, True, True, True, True, True, True, True, True],
    [False, False, True, True, True, True, True, False, False, False],
    [False, False, True, True, True, True, True, False, False, False],
])




test_2 = np.array([False, False, True, True, True, True, True, False, False, False])



ys, xs = np.where(test)
print(f'{ys = }')
print(f'{xs = }')
np.where(test_2)


In [2]:
image_path = '../test-images-mnz/castle_and_guards.jpg'
save_path = '../target'
number_colors = 16

create_paint_by_numbers(image_path, save_path, number_colors)

  0%|          | 0/218 [00:00<?, ?it/s]/var/folders/5_/vxqcbv5j3lxcvs8v3yhwhrkh0000gn/T/ipykernel_54472/760004499.py:159: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  font_size = font.getsize(str(number))
100%|██████████| 218/218 [00:04<00:00, 47.95it/s]

Paint-by-numbers image saved to ../target/paint_by_numbers_output.png
Legend saved to ../target/legend.txt


In [ ]:
np.where(np.array([
    True, True
]))